# Notebook to test new functions

In [ ]:
nest_dir = "neural_encoding_simulation_toolkit" 
test_images = "tutorial_images"

---

# Prepare all images

In [ ]:
from tqdm import tqdm
from torchvision import transforms as trn
import numpy as np
import os
from PIL import Image

images_dir = os.path.join(test_images)
images_list = os.listdir(images_dir)
images_list.sort()
    
images = []
for img in tqdm(images_list):
    img_dir = os.path.join(images_dir, img)
    img = Image.open(img_dir).convert('RGB')
    # Center crop the images to square format, and resize them
    transform = trn.Compose([
        trn.CenterCrop(min(img.size)),
        trn.Resize((227,227))
    ])
    img = transform(img)
    img = np.asarray(img)
    img = img.transpose(2,0,1)
    images.append(img)
images = np.asarray(images)

# Print the images dimensions
print('\n\nImages shape:')
print(images.shape)
print('(Batch size × 3 RGB Channels x Width x Height)')

---

# How to run the code in general:

In [ ]:
from nest import NEST

nest = NEST(nest_dir=nest_dir)

# List all available models and their versions
available_models = nest.list_models()
print(f"Available models: {available_models}")

# See what modalities are available
catalog = nest.get_model_catalog(print_format=True)
print(f"Model Catalog as Dict:: {catalog}")

In [ ]:
# Load Encoding Model
fwrf_model = nest.get_encoding_model("fmri-nsd-fwrf", 
                                     subject=1, 
                                     device="cpu",
                                     selection={"roi": "V1"})


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    device="auto")


fwrf_silico = nest.encode(fwrf_model, images)
eeg_silico, eeg_metadata = nest.encode(eeg_model, images,return_metadata=True)

---

# fwRF Model in more detail

In [ ]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)

# Choose a model of interest
model_id = "fmri-nsd-fwrf"  # Example model ID

# Get comprehensive model information
model_info = nest.describe(model_id)


In [ ]:
# Load Encoding Model
fwrf_model = nest.get_encoding_model("fmri-nsd-fwrf", 
                                     subject=1, 
                                     selection={"roi": "V1"})

In [ ]:
# Encode images
nest.encode(fwrf_model, images)

In [ ]:
# Encode images and return metadata
nest.encode(fwrf_model, images, return_metadata=True)

In [ ]:
nest.get_metadata_from_id("fmri-nsd-fwrf", 
                          subject=1,
                          roi="V1")

fwrf_model = nest.get_encoding_model("fmri-nsd-fwrf", 
                                     subject=1,
                                     roi="V1")
fwrf_model.get_metadata()

In [ ]:
# Just return metadata of model
fwrf_model.get_metadata()

--- 

# Testing EEG Model

In [ ]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1)

In [ ]:
nest.encode(eeg_model, images,return_metadata=True)

With Selection Parameter

In [ ]:
feeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    device="auto",
                                    selection={"channels": ['F7']})

eeg_silico = nest.encode(eeg_model, images)

print(eeg_silico.shape)

In [ ]:
vector = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    device="auto",
                                    selection={"channels": ['F7'],
                                               "timepoints": vector})

eeg_silico = nest.encode(eeg_model, images)

print(eeg_silico.shape)

---

### You can describe a model in two ways

In [ ]:
description = nest.describe("eeg-things_eeg_2-vit_b_32")

In [ ]:
description = eeg_model.describe()

# Check if Metadataretrieval works

In [ ]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1)
eeg_silico_total, eeg_metadata = nest.encode(eeg_model, images, return_metadata=True)

In [ ]:
channel = 8

print(eeg_metadata["eeg"]["ch_names"][channel])
print(eeg_silico_total.shape)
eeg_silico_total[:,:,channel,:].shape

In [ ]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    selection={"channels": [eeg_metadata["eeg"]["ch_names"][channel]]})
eeg_silico = nest.encode(eeg_model, images)

print(eeg_silico.shape)

np.all(eeg_silico == eeg_silico_total[:,:,channel,:])